In [1]:
from robotic import ry
import numpy as np
import time

###########################################PART-2 STARTS HERE##########################################

In [2]:
C = ry.Config()
C.addFile('cargobot.g')
C.addFile('maze.g')
C.addFile('cargo.g')


qHome = C.getJointState()
C.setJointState(qHome)

Get Final Pose with KOMO

In [3]:
komo = ry.KOMO(C, 1, 1, 2, True)
komo.addControlObjective([], 0, 1e-1)
komo.addControlObjective([], 2, 1e0)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq, [1e2])
komo.addObjective([1], ry.FS.positionDiff, ['l_panda_joint7', 'cargo_handle'], ry.OT.eq, [1e2])
komo.addObjective([1], ry.FS.jointState, [], ry.OT.eq, [1e2], [], order=1)

ret = ry.NLP_Solver(komo.nlp(), verbose=0).solve()
print(ret)
q = komo.getPath()
print('size of path:', q.shape)

qT = komo.getPath()[0]
C.setJointState(qT)
C.view(False, "IK solution")


-- WARNING:kin.cpp:getFrame:392(-1) cannot find frame named 'gripper'
STACK9 /bin/python3(_PyEval_EvalFrameDefault
STACK8 /bin/python3(+0x16e7bb) [0x5639a986a7bb]
STACK7 /bin/python3(_PyObject_MakeTpCall
STACK6 /bin/python3(+0x15fe0e) [0x5639a985be0e]
STACK5 /home/bornozlusapiens/.local/lib/python3.10/site-packages/robotic/ry.so(+0x7b93fd) [0x7fcea63b93fd]
STACK4 /home/bornozlusapiens/.local/lib/python3.10/site-packages/robotic/ry.so(+0x82ef7f) [0x7fcea642ef7f]
STACK3 /home/bornozlusapiens/.local/lib/python3.10/site-packages/robotic/ry.so(+0x7c0052) [0x7fcea63c0052]
STACK2 symbols2feature(FeatureSymbol, rai::Array<rai::String> const&, rai::Configuration const&, rai::ArrayDouble const&, rai::ArrayDouble const&, int)
STACK1 rai::Configuration::getFrameIDs(rai::Array<rai::String> const&) const
STACK0 rai::LogToken::~LogToken()


== ERROR:kin.cpp:getFrameIDs:423(-2) frame name 'gripper' doesn't exist


RuntimeError: kin.cpp:getFrameIDs:423(-2) frame name 'gripper' doesn't exist

RRT

In [8]:
#define a path finding problem
rrt = ry.PathFinder()
rrt.setProblem(C, [qHome], [qT])

for i in range(0, 2):
    ret = rrt.solve()
    print("rettttt: ", ret)
    path = ret.x
    print("path size:", path.shape[0])
    # display the path
    for t in range(0, path.shape[0] - 1):
        C.setJointState(path[t])
        C.view()
        time.sleep(0.002)

NameError: name 'qT' is not defined

Trajectory Optimization via KOMO

In [ ]:
C.setJointState(qHome)
komo2 = ry.KOMO(C, 1, path.shape[0], 2, True)
komo2.addControlObjective([], 0, 1e-1)
komo2.addControlObjective([], 2, 1e0)
komo2.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq, [1e2])
komo2.addObjective([1], ry.FS.positionDiff, ['base', 'goal_area'], ry.OT.eq, [1e2])
komo2.addObjective([1], ry.FS.jointState, [], ry.OT.eq, [1e1], [], order=1)
komo2.initWithPath_qOrg(path)

ret = ry.NLP_Solver(komo2.nlp(), verbose=0).solve()
q = komo2.getPath()
print('last path:', q)

# display the path for part 1
for i in range(0, q.shape[0]):
    C.setJointState(q[i])
    C.view(False, f'waypoints{i}')
    time.sleep(0.02)


###########################################PART-2 ENDS HERE##########################################